# **Laboratorio #08: Reconocimiento Descendente**

**Integrantes (Compilado):**

- Diego Alejandro Vásquez Aguilar. Documento: 1214743976.
- Sarai Restrepo Rodríguez. Documento: 1015066047.

## Enfoque 1: Analizador Descendente Recursivo (Calculadora con AST)



Este enfoque implementa una solución completa de calculadora con los siguientes componentes:

1.  **Lexer:** Convierte el texto en tokens (maneja números de varios dígitos).

2.  **Nodos AST:** Define la estructura del árbol (`NumNode`, `BinOpNode`, `UnaryOpNode`).

3.  **Parser:** Analizador descendente recursivo que consume tokens y construye el AST. Maneja operadores unarios (`-`).

4.  **Evaluador:** Recorre el AST para calcular el resultado final.

In [ ]:
from dataclasses import dataclass



@dataclass

class Token:

    type: str

    value: any

    pos: int



class LexerError(Exception):

    pass



class ParserError(Exception):

    pass



class Lexer:

    def __init__(self, text: str):

        self.text = text

        self.pos = 0



    def tokenize(self):

        tokens = []

        while self.pos < len(self.text):

            char = self.text[self.pos]

            if char.isspace():

                self.pos += 1

                continue

            if char.isdigit():

                start = self.pos

                while self.pos < len(self.text) and self.text[self.pos].isdigit():

                    self.pos += 1

                value = int(self.text[start:self.pos])

                tokens.append(Token("NUM", value, start))

                continue

            if char == '+':

                tokens.append(Token("PLUS", char, self.pos))

                self.pos += 1

                continue

            if char == '-':

                tokens.append(Token("MINUS", char, self.pos))

                self.pos += 1

                continue

            if char == '*':

                tokens.append(Token("MUL", char, self.pos))

                self.pos += 1

                continue

            if char == '/':

                tokens.append(Token("DIV", char, self.pos))

                self.pos += 1

                continue

            if char == '(':

                tokens.append(Token("LPAREN", char, self.pos))

                self.pos += 1

                continue

            if char == ')':

                tokens.append(Token("RPAREN", char, self.pos))

                self.pos += 1

                continue

            raise LexerError(f"Carácter inválido '{char}' en la posición {self.pos}")

        tokens.append(Token("EOF", None, self.pos))

        return tokens



@dataclass

class NumNode:

    value: int



@dataclass

class BinOpNode:

    left: any

    op: str

    right: any



@dataclass

class UnaryOpNode:

    op: str

    expr: any



class Parser:

    def __init__(self, tokens):

        self.tokens = tokens

        self.pos = 0

        self.current = self.tokens[self.pos]



    def error(self, expected: str):

        if self.current.type == "EOF":

            raise ParserError(f"Error de sintaxis: se esperaba {expected}, pero se encontró fin de expresión.")

        else:

            raise ParserError(

                f"Error de sintaxis cerca de posición {self.current.pos}: "

                f"se esperaba {expected}, pero se encontró '{self.current.type}'."

            )



    def advance(self):

        self.pos += 1

        if self.pos < len(self.tokens):

            self.current = self.tokens[self.pos]



    def eat(self, token_type: str):

        if self.current.type == token_type:

            self.advance()

        else:

            self.error(token_type)



    def parse(self):

        node = self.expr()

        if self.current.type != "EOF":

            self.error("EOF")

        return node



    def expr(self):

        node = self.term()

        while self.current.type in ("PLUS", "MINUS"):

            op = self.current.type

            self.advance()

            right = self.term()

            node = BinOpNode(left=node, op=op, right=right)

        return node



    def term(self):

        node = self.factor()

        while self.current.type in ("MUL", "DIV"):

            op = self.current.type

            self.advance()

            right = self.factor()

            node = BinOpNode(left=node, op=op, right=right)

        return node



    def factor(self):

        tok = self.current

        if tok.type == "NUM":

            self.eat("NUM")

            return NumNode(tok.value)

        if tok.type == "LPAREN":

            self.eat("LPAREN")

            node = self.expr()

            if self.current.type != "RPAREN":

                self.error("')'")

            self.eat("RPAREN")

            return node

        if tok.type == "MINUS":

            self.eat("MINUS")

            expr = self.factor()

            return UnaryOpNode(op="MINUS", expr=expr)

        self.error("número, '(', o '-' unario")



class Evaluator:

    def eval(self, node):

        if isinstance(node, NumNode):

            return node.value

        if isinstance(node, UnaryOpNode):

            val = self.eval(node.expr)

            if node.op == "MINUS":

                return -val

            raise ValueError(f"Operador unario desconocido {node.op}")

        if isinstance(node, BinOpNode):

            left = self.eval(node.left)

            right = self.eval(node.right)

            if node.op == "PLUS":

                return left + right

            if node.op == "MINUS":

                return left - right

            if node.op == "MUL":

                return left * right

            if node.op == "DIV":

                if right == 0:

                    raise ZeroDivisionError("Error: división por cero.")

                return left / right

            raise ValueError(f"Operador binario desconocido {node.op}")

        raise ValueError("Nodo AST desconocido")



def analizar_expresion(texto: str):

    lexer = Lexer(texto)

    tokens = lexer.tokenize()

    parser = Parser(tokens)

    ast = parser.parse()

    evaluator = Evaluator()

    resultado = evaluator.eval(ast)

    return tokens, ast, resultado

## Enfoque 2: Analizador LL(1) Dirigido por Tabla (Reconocimiento)



Este enfoque implementa el algoritmo clásico de análisis LL(1) usando una pila y una tabla.



**Gramática LL(1) (Sin unario):**

```

E  → T E'

E' → + T E' | - T E' | ε

T  → F T'

T' → * F T' | / F T' | ε

F  → ( E ) | num | id

```

**Salida:** La función `analizar_cadena` genera un DataFrame de Pandas que muestra el estado de la **Pila**, la **Entrada** y la **Acción** en cada paso, cumpliendo el requisito de un informe gráfico paso a paso.

In [ ]:
import pandas as pd



GRAMATICA = {

    "E":  ["T E'"],

    "E'": ["+ T E'", "- T E'", "ε"],

    "T":  ["F T'"],

    "T'": ["* F T'", "/ F T'", "ε"],

    "F":  ["( E )", "num", "id"],

}



TABLA_LL1 = {

    "E":  {"(": "T E'", "num": "T E'", "id": "T E'"},

    "E'": {"+": "+ T E'", "-": "- T E'", ")": "ε", "$": "ε"},

    "T":  {"(": "F T'", "num": "F T'", "id": "F T'"},

    "T'": {"+": "ε", "-": "ε", "*": "* F T'", "/": "/ F T'", ")": "ε", "$": "ε"},

    "F":  {"(": "( E )", "num": "num", "id": "id"},

}



def analizar_cadena(cadena: str):

    simbolos = _expr_a_simbolos_ll1(cadena)

    if simbolos is None:

      raise ValueError("Cadena contiene símbolos inválidos para la gramática LL(1)")

    entrada = simbolos + ["$"]

    pila = ["$", "E"]

    pasos = []

    i = 0

    while True:

        cima = pila[-1]

        simbolo = entrada[i]

        pila_str = " ".join(pila)

        entrada_str = " ".join(entrada[i:])

        if cima == "$" and simbolo == "$":

            pasos.append({"Paso": len(pasos) + 1, "Pila": pila_str, "Entrada": entrada_str, "Acción": "ACCEPTED ✅"})

            break

        if cima == simbolo:

            pasos.append({"Paso": len(pasos) + 1, "Pila": pila_str, "Entrada": entrada_str, "Acción": f"match '{simbolo}'"})

            pila.pop()

            i += 1

            continue

        if cima in TABLA_LL1:

            regla = TABLA_LL1[cima].get(simbolo)

            if regla is None:

                pasos.append({"Paso": len(pasos) + 1, "Pila": pila_str, "Entrada": entrada_str, "Acción": f"❌ ERROR: no hay producción para ({cima}, {simbolo})"})

                break

            pasos.append({"Paso": len(pasos) + 1, "Pila": pila_str, "Entrada": entrada_str, "Acción": f"aplicar {cima} → {regla}"})

            pila.pop()

            if regla != "ε":

                for s in reversed(regla.split()):

                    pila.append(s)

            continue

        pasos.append({"Paso": len(pasos) + 1, "Pila": pila_str, "Entrada": entrada_str, "Acción": f"❌ ERROR: se esperaba '{cima}' pero se encontró '{simbolo}'"})

        break

    return pd.DataFrame(pasos)



def _expr_a_simbolos_ll1(expr: str):

    tokens = []

    i = 0

    while i < len(expr):

        c = expr[i]

        if c.isspace():

            i += 1

            continue

        if c.isdigit():

            j = i

            while j < len(expr) and expr[j].isdigit():

                j += 1

            tokens.append("num")

            i = j

            continue

        if c.isalpha() or c == "_":

            j = i

            while j < len(expr) and (expr[j].isalnum() or expr[j] == "_"):

                j += 1

            tokens.append("id")

            i = j

            continue

        if c in "+-*/()":

            tokens.append(c)

            i += 1

            continue

        return None

    return tokens



def _tiene_unario_no_ll1(simbolos_ll1):

    if not simbolos_ll1:

        return False

    ops = {"+", "-", "*", "/"}

    for idx, s in enumerate(simbolos_ll1):

        if s == "-":

            if idx == 0:

                return True

            if simbolos_ll1[idx-1] in ops or simbolos_ll1[idx-1] == "(":

                return True

    return False

ModuleNotFoundError: No module named 'pandas'

## 🚀 Interfaz Gráfica (Pruebas en Modo Gráfico)



Esta interfaz gráfica unifica ambos enfoques en dos pestañas:

1.  **Calculadora:** Utiliza el Analizador Descendente Recursivo (Enfoque 1) para evaluar expresiones complejas, incluyendo unarios.

2.  **Reconocimiento LL(1):** Utiliza el Analizador por Tabla (Enfoque 2) para generar el informe de análisis paso a paso.

In [ ]:
import ipywidgets as widgets

from IPython.display import display, clear_output

import pandas as pd



title = widgets.HTML("<h2>Analizador Sintáctico — Práctica #8</h2>")



# Pestaña 1: Calculadora

in_calc = widgets.Text(value='', placeholder='Ej: -5 + 3*(2-8)', description='Expresión:', layout=widgets.Layout(width='85%'))

btn_calc = widgets.Button(description='Analizar y Evaluar', button_style='info', icon='play')

out_calc = widgets.Output(layout={'border': '1px solid #ccc', 'max_height': '380px', 'overflow': 'auto'})



def on_click_calc(_):

    with out_calc:

        clear_output()

        expr = in_calc.value

        if not expr.strip():

            print("Por favor ingresa una expresión.")

            return

        try:

            tokens, ast, resultado = analizar_expresion(expr)

            print("=== TOKENS ===")

            for t in tokens:

                print(f"{t.type:<6}  valor={t.value}  pos={t.pos}")

            print("\n=== AST (Formato Árbol) ===")

            for line in ast_pretty(ast):

                print(line)

            print("\n=== RESULTADO ===")

            print("✅ Expresión válida.")

            print("Resultado =", resultado)

        except LexerError as le:

            print("=== ❌ ERROR LÉXICO ===\n" + str(le))

        except ParserError as pe:

            print("=== ❌ ERROR SINTÁCTICO ===\n" + str(pe))

        except ZeroDivisionError as zde:

            print("=== ❌ ERROR EN EJECUCIÓN ===\n" + str(zde))

        except Exception as e:

            print("=== ❌ ERROR DESCONOCIDO ===\n" + str(e))



btn_calc.on_click(on_click_calc)

box_calc = widgets.VBox([in_calc, btn_calc, out_calc])



# Pestaña 2: Reconocimiento LL(1)

hint_ll1 = widgets.HTML(

    """<b>Nota:</b> La gramática LL(1) reconoce <code>num</code>, <code>id</code>, <code>+</code>, <code>-</code>, <code>*</code>, <code>/</code>, <code>(</code>, <code>)</code>.<br>

    Números e identificadores se normalizan a <code>num</code>/<code>id</code> automáticamente.<br>

    El signo <b>unario</b> <code>-</code> no está contemplado explícitamente en la tabla; se sugiere usar (0 - x)."""

)

in_ll1 = widgets.Text(value='id * ( id + num )', placeholder='Ej: id * ( id + num * id )', description='Entrada:', layout=widgets.Layout(width='85%'))

btn_ll1 = widgets.Button(description='Generar Tabla', button_style='info', icon='table')

out_ll1 = widgets.Output(layout={'border': '1px solid #ccc', 'max_height': '380px', 'overflow': 'auto'})



def on_click_ll1(_):

    with out_ll1:

        clear_output()

        expr = in_ll1.value

        if not expr.strip():

            print("Por favor ingresa una cadena.")

            return

        simbolos = _expr_a_simbolos_ll1(expr)

        if simbolos is None:

            print("⚠ La cadena contiene símbolos fuera de la gramática LL(1).")

            print("   Solo se admiten: números, identificadores, + - * / ( )")

            return

        if _tiene_unario_no_ll1(simbolos):

            print("ℹ Se detectó '-' unario. La tabla LL(1) de clase no lo contempla.")

            print("   Sugerencia: reescribir como (0 - x) o eliminar el unario para esta simulación.")

            return

        print("Entrada normalizada LL(1):", ' '.join(simbolos))

        try:

            df = analizar_cadena(expr)

            estado = "" 

            if "Acción" in df.columns and df["Acción"].astype(str).str.contains("ACCEPTED").any():

                estado = "✅ ACEPTADA"

            elif df["Acción"].astype(str).str.contains("ERROR").any():

                estado = "❌ RECHAZADA / ERROR"

            else:

                estado = "ℹ Revisar pasos: no se encontró ACCEPTED ni ERROR explícito."

            print("Estado:", estado)

            print("\n=== REPORTE LL(1) (PASO A PASO) ===")

            display(df)

        except Exception as e:

            print("✘ Error en reconocimiento LL(1):", e)



btn_ll1.on_click(on_click_ll1)

box_ll1 = widgets.VBox([hint_ll1, in_ll1, btn_ll1, out_ll1])



# Pestaña 3: Paso a Paso Interactivo LL(1)

in_ll1_step = widgets.Text(value='a + b * ( c - 7 )', placeholder='Expresión para simular LL(1)', description='Entrada:', layout=widgets.Layout(width='85%'))

btn_run_steps = widgets.Button(description='Preparar Simulación', button_style='warning', icon='cogs')

slider_step = widgets.IntSlider(value=1, min=1, max=1, step=1, description='Paso:', continuous_update=False)

out_steps = widgets.Output(layout={'border': '1px solid #ccc', 'max_height': '380px', 'overflow': 'auto'})



_cached_steps = []



def preparar_simulacion(_):

    global _cached_steps

    with out_steps:

        clear_output()

        expr = in_ll1_step.value

        simbolos = _expr_a_simbolos_ll1(expr)

        if simbolos is None:

            print("⚠ Símbolos inválidos para LL(1).")

            return

        if _tiene_unario_no_ll1(simbolos):

            print("ℹ '-' unario detectado. Reescribe como (0 - x) si deseas simular.")

            return

        try:

            pasos, ok = simular_ll1_pasos(expr)

            _cached_steps = pasos

            slider_step.max = len(pasos)

            slider_step.value = 1

            print(f"Simulación preparada. Total pasos: {len(pasos)}. Usa el slider para avanzar.")

            mostrar_paso()

        except Exception as e:

            print("Error preparando simulación:", e)



def mostrar_paso(_=None):

    with out_steps:

        clear_output(wait=True)

        if not _cached_steps:

            print("Primero prepara la simulación.")

            return

        paso_idx = slider_step.value - 1

        paso = _cached_steps[paso_idx]

        print(f"Paso {paso['Paso']} de {len(_cached_steps)}")

        print(f"Pila    : {paso['Pila']}")

        print(f"Entrada : {paso['Entrada']}")

        print(f"Acción  : {paso['Acción']}")

        if paso_idx + 1 < len(_cached_steps):

            print("\nPróximo:")

            print(f"  → { _cached_steps[paso_idx+1]['Acción'] }")

        if paso_idx + 2 < len(_cached_steps):

            print(f"  → { _cached_steps[paso_idx+2]['Acción'] }")



btn_run_steps.on_click(preparar_simulacion)

slider_step.observe(mostrar_paso, names='value')

box_ll1_steps = widgets.VBox([in_ll1_step, btn_run_steps, slider_step, out_steps])



tabs = widgets.Tab(children=[box_calc, box_ll1, box_ll1_steps])

tabs.set_title(0, 'Calculadora (Recursiva)')

tabs.set_title(1, 'Reconocimiento LL(1)')

tabs.set_title(2, 'Paso a Paso LL(1)')



display(title, tabs)

## Mejoras respecto a los documentos base



- Soporte de números de varios dígitos en todos los módulos.

- Mensajes de error más claros, con posición y sugerencias.

- Informe LL(1) paso a paso con DataFrame y nuevo visor paso a paso interactivo.

- Evaluación segura en la calculadora (sin usar `eval` para la calculadora principal).

- Pestañas separadas: Calculadora, Reconocimiento LL(1) y Paso a Paso LL(1).

In [ ]:
def ast_pretty(node, indent=""):

    lines = []

    if isinstance(node, NumNode):

        lines.append(f"{indent}Num({node.value})")

    elif isinstance(node, UnaryOpNode):

        lines.append(f"{indent}{node.op}")

        lines.extend(ast_pretty(node.expr, indent + "  "))

    elif isinstance(node, BinOpNode):

        lines.append(f"{indent}{node.op}")

        lines.extend(ast_pretty(node.left, indent + "  "))

        lines.extend(ast_pretty(node.right, indent + "  "))

    else:

        lines.append(f"{indent}<desconocido>")

    return lines



def simular_ll1_pasos(cadena: str):

    simbolos = _expr_a_simbolos_ll1(cadena)

    if simbolos is None:

        raise ValueError("Cadena contiene símbolos fuera de la gramática LL(1)")

    entrada = simbolos + ["$"]

    pila = ["$", "E"]

    pasos = []

    i = 0

    while True:

        cima = pila[-1]

        simbolo = entrada[i]

        pila_str = " ".join(pila)

        entrada_str = " ".join(entrada[i:])

        if cima == "$" and simbolo == "$":

            pasos.append({"Paso": len(pasos) + 1, "Pila": pila_str, "Entrada": entrada_str, "Acción": "ACCEPTED ✅"})

            return pasos, True

        if cima == simbolo:

            pasos.append({"Paso": len(pasos) + 1, "Pila": pila_str, "Entrada": entrada_str, "Acción": f"match '{simbolo}'"})

            pila.pop(); i += 1

            continue

        if cima in TABLA_LL1:

            regla = TABLA_LL1[cima].get(simbolo)

            if regla is None:

                pasos.append({"Paso": len(pasos) + 1, "Pila": pila_str, "Entrada": entrada_str, "Acción": f"❌ ERROR: no hay producción para ({cima}, {simbolo})"})

                return pasos, False

            pasos.append({"Paso": len(pasos) + 1, "Pila": pila_str, "Entrada": entrada_str, "Acción": f"aplicar {cima} → {regla}"})

            pila.pop()

            if regla != "ε":

                for s in reversed(regla.split()):

                    pila.append(s)

            continue

        pasos.append({"Paso": len(pasos) + 1, "Pila": pila_str, "Entrada": entrada_str, "Acción": f"❌ ERROR: se esperaba '{cima}' pero se encontró '{simbolo}'"})

        return pasos, False

In [5]:
# Test rápido de la calculadora sin pandas

expr = "2+3*4"

try:

    tokens, ast, resultado = analizar_expresion(expr)

    print("Expr:", expr)

    print("Resultado esperado 14, obtenido:", resultado)

except Exception as e:

    print("Error:", e)

Expr: 2+3*4
Resultado esperado 14, obtenido: 14


## Gramática, Conjuntos FIRST y FOLLOW (Referencia)



```text

Gramática

  E  -> T E'

  E' -> + T E' | - T E' | ε

  T  -> F T'

  T' -> * F T' | / F T' | ε

  F  -> ( E ) | num | id



FIRST

  FIRST(E)  = { (, num, id }

  FIRST(E') = { +, -, ε }

  FIRST(T)  = { (, num, id }

  FIRST(T') = { *, /, ε }

  FIRST(F)  = { (, num, id }



FOLLOW

  FOLLOW(E)  = { ), $ }

  FOLLOW(E') = { ), $ }

  FOLLOW(T)  = { +, -, ), $ }

  FOLLOW(T') = { +, -, ), $ }

  FOLLOW(F)  = { *, /, +, -, ), $ }

```



Esta gramática es LL(1) porque para cada no terminal las producciones tienen conjuntos FIRST disjuntos, y cuando una producción incluye ε, su FIRST y FOLLOW son disjuntos.


In [ ]:
from IPython.display import display, clear_output

import ipywidgets as widgets

import pandas as pd



class GIC:

    def __init__(self):

        self.tabla_ll1 = {

            'E':  {'(': ['T','E\''], 'num': ['T','E\''], 'id': ['T','E\'']},

            "E'": {'+': ['+','T','E\''], '-': ['-','T','E\''], ')': ['ε'], '$': ['ε']},

            'T':  {'(': ['F','T\''], 'num': ['F','T\''], 'id': ['F','T\'']},

            "T'": {'*': ['*','F','T\''], '/': ['/','F','T\''], '+': ['ε'], '-': ['ε'], ')': ['ε'], '$': ['ε']},

            'F':  {'(': ['(','E',')'], 'num': ['num'], 'id': ['id']}

        }



    def tokenizar(self, cadena: str):

        tokens = []

        i = 0

        while i < len(cadena):

            c = cadena[i]

            if c.isspace():

                i += 1; continue

            if c.isdigit():

                j = i

                while j < len(cadena) and cadena[j].isdigit():

                    j += 1

                tokens.append('num'); i = j; continue

            if c.isalpha() or c == '_':

                j = i

                while j < len(cadena) and (cadena[j].isalnum() or cadena[j] == '_'):

                    j += 1

                tokens.append('id'); i = j; continue

            if c in '+-*/()':

                tokens.append(c); i += 1; continue

            return None

        tokens.append('$')

        return tokens



    def produccion(self, no_terminal, terminal):

        try:

            prod = self.tabla_ll1[no_terminal][terminal]

            return [] if prod == ['ε'] else list(reversed(prod))

        except KeyError:

            return None



    def reconocer(self, cadena: str):

        entrada = self.tokenizar(cadena)

        if entrada is None:

            return [], False, 'símbolo inválido'

        pila = ['$','E']

        pasos = []

        while pila:

            tope = pila[-1]

            token = entrada[0] if entrada else '$'

            pasos.append({'Pila': ' '.join(pila), 'Entrada': ' '.join(entrada)})

            if tope == token:

                pila.pop(); entrada.pop(0); continue

            if tope not in self.tabla_ll1:

                return pasos, False, token

            prod_rev = self.produccion(tope, token)

            if prod_rev is None:

                return pasos, False, token

            pila.pop(); pila.extend(prod_rev)

        if entrada:

            return pasos, False, entrada[0]

        return pasos, True, None



gic_alt = GIC()

input_alt = widgets.Text(placeholder='Ej: a + b * (c - 25)', description='Expr:', layout=widgets.Layout(width='70%'))

btn_alt = widgets.Button(description='Reconocer LL(1) (Alt)', button_style='primary', icon='check')

out_alt = widgets.Output(layout={'border':'1px solid #999','padding':'4px','max_height':'320px','overflow':'auto'})



def on_alt(_):

    with out_alt:

        clear_output()

        expr = input_alt.value

        if not expr.strip():

            print('Ingresa una expresión.'); return

        pasos, ok, err = gic_alt.reconocer(expr)

        df = pd.DataFrame(pasos)

        display(df)

        if ok:

            print('\n✅ Cadena aceptada por LL(1) alternativo.')

        else:

            print(f"\n❌ Cadena no aceptada. Problema cerca de: {err}")

        if ok and all(t not in expr for t in ['id']):

            try:

                if all(ch.isdigit() or ch in '+-*/() ' for ch in expr):

                    _, _, res = analizar_expresion(expr)

                    print(f'Resultado calculadora = {res}')

            except Exception as e:

                print('Evaluación opcional falló:', e)



btn_alt.on_click(on_alt)

display(widgets.VBox([widgets.HTML('<h3>Modo alternativo LL(1) (GIC)</h3>'), input_alt, btn_alt, out_alt]))

In [ ]:
# Prueba integrada: calcular y reconocer '2+3*4'

expr_test = '2+3*4'

print('Expresión:', expr_test)

tokens, ast, resultado = analizar_expresion(expr_test)

print('Resultado parser recursivo =', resultado)

gic_steps, ok, err = gic_alt.reconocer(expr_test)

print('LL(1) alternativo acepta?', ok, '| error:', err)

print('Total pasos LL(1):', len(gic_steps))

In [ ]:
# Prueba específica de las expresiones solicitadas

print('='*60)

print('PRUEBA 1: 3+4*(5 + 4-3)')

print('='*60)

expr1 = '3+4*(5 + 4-3)'

try:

    tokens1, ast1, resultado1 = analizar_expresion(expr1)

    print(f'✅ Expresión válida')

    print(f'Resultado = {resultado1}')

    print('\nTokens:')

    for t in tokens1:

        if t.type != 'EOF':

            print(f'  {t.type:<8} valor={t.value}')

except LexerError as e:

    print(f'❌ ERROR LÉXICO: {e}')

except ParserError as e:

    print(f'❌ ERROR SINTÁCTICO: {e}')

except Exception as e:

    print(f'❌ ERROR: {e}')



print('\n' + '='*60)

print('PRUEBA 2: 3+4*/(5 + 4-3)')

print('='*60)

expr2 = '3+4*/(5 + 4-3)'

try:

    tokens2, ast2, resultado2 = analizar_expresion(expr2)

    print(f'✅ Expresión válida')

    print(f'Resultado = {resultado2}')

except LexerError as e:

    print(f'❌ ERROR LÉXICO: {e}')

except ParserError as e:

    print(f'❌ ERROR SINTÁCTICO: {e}')

except Exception as e:

    print(f'❌ ERROR: {e}')



print('\n' + '='*60)

print('EXPLICACIÓN:')

print('='*60)

print('Expr 1: Válida - operadores y paréntesis correctos')

print('Expr 2: Inválida - operadores */ juntos (error sintáctico)')